<a href="https://colab.research.google.com/github/poovendhiranr/Test_public_spark/blob/main/Analysis_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip3 install pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType,ArrayType
import math 
import pyspark.sql.functions as f
from pyspark.sql import types 
from pyspark.sql import Window
spark = SparkSession.builder.master("local[*]") \
                    .appName('Analysis') \
                    .getOrCreate()



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Summary Report (All Time)

In [53]:
df=spark.read.csv("/content/datafiles",header=True)
meta=spark.read.csv("/content/meta/symbol_metadata.csv",header=True)
df1=df.withColumn("filename",f.input_file_name())
df2=df1.withColumn("filename",f.regexp_replace(f.split(df1['filename'], '/').getItem(5),'.csv',''))
df3=df2.join(meta,f.trim(df2.filename)==f.trim(meta.Symbol),"inner")
df3.orderBy('timestamp') \
    .groupBy("Sector") \
    .agg(f.round(f.avg('open'),2).alias('Avg_OpenPrice'), \
         f.round(f.avg('close'),2).alias('Avg_ClosePrice'), \
         f.round(f.max('high'),2).alias('Max_high'), \
         f.round(f.min('low'),2).alias('Min_low'), \
         f.round(f.avg('volume'),2).alias('Avg_volume')).show()


+--------------------+-------------+--------------+--------+-------+-------------+
|              Sector|Avg_OpenPrice|Avg_ClosePrice|Max_high|Min_low|   Avg_volume|
+--------------------+-------------+--------------+--------+-------+-------------+
|ENERGY & TRANSPOR...|        17.56|         17.56|   99.99|    0.2|   4669611.43|
|             FINANCE|         19.6|         19.61|    9.99|   10.0|     10065.09|
|       LIFE SCIENCES|         45.3|         45.31|   99.93|  10.29|    1772380.1|
|       MANUFACTURING|        32.83|         32.84|   92.32|  10.09|   5396276.95|
|REAL ESTATE & CON...|        10.18|         10.17|    9.99|   10.0|    212976.18|
|          TECHNOLOGY|        72.61|         72.61|   99.99|   0.09|1.149873898E7|
|    TRADE & SERVICES|        58.67|         58.69|    99.9|  100.0|    2475920.9|
+--------------------+-------------+--------------+--------+-------+-------------+



Aggregation summary details for a given sector(s) over the time period.

In [73]:
#------------------input---------------------------------
dt=[('2021-01-01','2022-05-26',['TECHNOLOGY','FINANCE'])]
#----------------------------------------------------------
deptSchema = StructType([       
    StructField('dept_name', StringType(), True),
    StructField('dept_id', StringType(), True),
    StructField('array_valuer',ArrayType(StringType(),False)  ,True)
])
unioned_df = None
df_filter  =spark.createDataFrame(data=dt,schema=deptSchema)
df1_filter =df_filter.withColumn("sector_value",f.explode(f.col('array_valuer'))).drop('array_valuer')
dd_c={x[2]:[x[0],x[1]] for x in df1_filter.rdd.collect()}  
for i,k in dd_c.items():
  sector_value= i
  start_date=k[0]
  end_date  =k[1]
  df_temp   =df3.filter((f.col("timestamp")>=start_date) & (f.col("timestamp")<=end_date)&(f.col("Sector")==sector_value))  
  
  if not unioned_df:
        unioned_df = df_temp
  else:
        unioned_df = unioned_df.union(df_temp)
agg_df=unioned_df.orderBy('timestamp') \
          .groupBy("Sector") \
          .agg(f.round(f.avg('open'),2).alias('Avg_OpenPrice'), \
           f.round(f.avg('close'),2).alias('Avg_ClosePrice'), \
           f.round(f.max('high'),2).alias('Max_high'), \
           f.round(f.min('low'),2).alias('Min_low'), \
           f.round(f.avg('volume'),2).alias('Avg_volume'))

agg_df.show()

+----------+-------------+--------------+--------+-------+-------------+
|    Sector|Avg_OpenPrice|Avg_ClosePrice|Max_high|Min_low|   Avg_volume|
+----------+-------------+--------------+--------+-------+-------------+
|   FINANCE|        24.83|         24.87|    35.0|  14.38|     25198.98|
|TECHNOLOGY|        78.96|         78.99|    99.9| 100.11|2.413942584E7|
+----------+-------------+--------------+--------+-------+-------------+



Aggregation summary details for all the symbol(s) in a given sector over
the time period.

In [74]:
#------------------input---------------------------------
dt=[('2021-01-01','2022-05-26',['TECHNOLOGY','FINANCE'])]
#----------------------------------------------------------
deptSchema = StructType([       
    StructField('dept_name', StringType(), True),
    StructField('dept_id', StringType(), True),
    StructField('array_valuer',ArrayType(StringType(),False)  ,True)
])
unioned_df1 = None
df_filter1  =spark.createDataFrame(data=dt,schema=deptSchema)
df1_filter1 =df_filter.withColumn("sector_value",f.explode(f.col('array_valuer'))).drop('array_valuer')
dd_c1={x[2]:[x[0],x[1]] for x in df1_filter.rdd.collect()}  
for i,k in dd_c1.items():
  sector_value= i
  start_date=k[0]
  end_date  =k[1]
  df_temp   =df3.filter((f.col("timestamp")>=start_date) & (f.col("timestamp")<=end_date)&(f.col("Sector")==sector_value))  
  if not unioned_df:
        unioned_df = df_temp
  else:
        unioned_df = unioned_df.union(df_temp)
agg_df1=unioned_df.orderBy('timestamp') \
          .groupBy("Symbol","Name") \
          .agg(f.round(f.avg('open'),2).alias('Avg_OpenPrice'), \
           f.round(f.avg('close'),2).alias('Avg_ClosePrice'), \
           f.round(f.max('high'),2).alias('Max_high'), \
           f.round(f.min('low'),2).alias('Min_low'), \
           f.round(f.avg('volume'),2).alias('Avg_volume'))
agg_df1.show()          
  

+------+--------------------+-------------+--------------+--------+-------+-------------+
|Symbol|                Name|Avg_OpenPrice|Avg_ClosePrice|Max_high|Min_low|   Avg_volume|
+------+--------------------+-------------+--------------+--------+-------+-------------+
|  AAPL|           Apple Inc|       145.34|         145.4|  182.94| 116.21|9.132503343E7|
|  ACNB|    ACNB Corporation|        29.02|         29.04|    35.0|  24.06|     21827.67|
|  BCML|         BayCom Corp|        18.36|         18.37|   22.73|  14.38|     34451.94|
|  DELL|Dell Technologies...|        82.81|         82.87|    99.9| 100.11|   3121529.36|
|  MYFW|First Western Fin...|         27.1|         27.21|    34.2|   17.3|     19317.34|
|  NTAP|          NetApp Inc|        81.26|         81.28|   96.82|  58.83|   1708967.17|
|  QMCO| Quantum Corporation|         6.42|          6.41|    9.47|   2.36|    402173.39|
+------+--------------------+-------------+--------------+--------+-------+-------------+

